# Central RL control notebook

In [1]:
import sys
from pymgrid import MicrogridGenerator as mg
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from IPython.display import display,clear_output
import pandas as pd
from Storage import StorageSuite as st
from pathlib import Path
from math import e
from time import sleep

Time span constants

In [2]:
YEAR = 31579200
MONTH = 2570400
WEEK = 604800
DAY = 86400
HOUR = 3600
MINUTE = 60

### Generate Microgrids

In [3]:
n_grids = 2
env_list = []

for i in range(n_grids):
    loaderino = np.random.randint(1E5,6E5)
    print(loaderino)
    env_list.append(st(r'C:\Users\thesu\Desktop\Research 2022 Microgrid\Research code repository\Micro-Grid-ML-algo\data\dummy_data_v1.csv',load=loaderino))
env = mg.MicrogridGenerator(storage_suite_list=env_list)
env.generate_microgrid(verbose= True, interpolate= True)
clear_output()

In [4]:
env.print_mg_parameters()

,load,load_size,cost_loss_load,cost_overgeneration,cost_co2,PV_rated_power,grid_weak,grid_power_import,grid_power_export,genset_polynom_order,genset_polynom_0,genset_polynom_1,genset_polynom_2,genset_rated_power,genset_pmin,genset_pmax,fuel_cost,genset_co2
0,349342,349342,10,1,0.1,216592.04,0,698684,698684,3,8.962852,0.699991,0.071430,388158,0.05,0.9,0.4,2
1,377869,377869,10,1,0.1,294737.82,1,755738,755738,3,3.992596,0.578001,0.058802,419855,0.05,0.9,0.4,2


In [5]:
mg0 = env.microgrids[0]
mg1 = env.microgrids[1]

In [6]:
# print(li_ion1.power) # 40 W
# print(flow1.power) # 120 W
# print(flywheel1.power) # 21.5 KW

In [7]:
# li_ion1,flow1,flywheel1 = mg0.ss.unpack()
# pv = round(mg0.pv)
# load = round(mg0.load)

# # li_ion,flow,flywheel = mg0.ss.unpack()
# pv = abs(round(mg0.pv))
# load = abs(round(mg0.load))
# undergen_net_load = abs(load-pv) 
# overgen_net_load = pv-load

# li_ion_charge_max = li_ion1.power
# li_ion_discharge_max = li_ion1.power
# li_ion_peak = li_ion1.peak_discharge
# li_ion_soc = li_ion1.soc
# li_ion_capa_to_charge = li_ion1.capa_to_charge
# li_ion_capa_to_discharge = li_ion1.capa_to_discharge

# flow_charge_max = flow1.power
# flow_discharge_max = flow1.power
# flow_peak = flow1.peak_discharge
# flow_soc = flow1.soc
# flow_capa_to_charge = flow1.capa_to_charge
# flow_capa_to_discharge = flow1.capa_to_discharge

# flywheel_charge_max = flywheel1.power
# flywheel_discharge_max = flywheel1.power
# flywheel_peak = flywheel1.peak_discharge
# flywheel_soc = flywheel1.soc
# flywheel_capa_to_charge = flywheel1.capa_to_charge
# flywheel_capa_to_discharge = flywheel1.capa_to_discharge
# flywheel1.soc = flywheel1.MIN_SOC
# control_dict = {    'pv_consummed': min(pv,load),
#                                 'li_charge': 0,
#                                 'li_discharge': 0,
#                                 'flow_charge': 0,
#                                 'flow_discharge': 0,
#                                 'flywheel_charge': flywheel_charge_max,
#                                 'flywheel_discharge': 0,
#                                 'grid_import': 0,
#                                 'grid_export': 0,
#                                 'genset': 0
#                             }
# print(control_dict)
# mg0.run(control_dict)

In [8]:
# print(flywheel1.soc)

In [9]:
# breaker()

In [10]:
# Grid is up

    # PV is above load

    # PV is below load


# Grid is down

    # PV is above load

    # PV is below load
        


# Stock control dict

# control_dict = {            'pv_consummed': min(pv,load),
#                             'li_charge': 0,
#                             'li_discharge': 0,
#                             'flow_charge': 0,
#                             'flow_discharge': 0,
#                             'flywheel_charge': 0,
#                             'flywheel_discharge': 0,
#                             'grid_import': 0,
#                             'grid_export':0,
#                             'genset': 0
#                            }

def actions_agent(mg0, action):
    li_ion,flow,flywheel = mg0.ss.unpack()
    pv =                        round(mg0.pv)
    load =                      round(mg0.load)
    undergen_net_load =         load-pv
    overgen_net_load =          pv-load

    li_ion_charge_max =          abs(li_ion.power)
    li_ion_discharge_max =       abs(li_ion.power)
    li_ion_soc =                 abs(li_ion.soc)
    flow_charge_max =            abs(flow.power)
    flow_discharge_max =         abs(flow.power)
    flow_soc =                   abs(flow.soc)
    flywheel_charge_max =        abs(flywheel.power)
    flywheel_discharge_max =     abs(flywheel.power)
    flywheel_soc =               abs(flywheel.soc)

    # li_ion_peak =                abs(li_ion.peak_discharge)
    # li_ion_capa_to_charge =      abs(li_ion.capa_to_charge)
    # li_ion_capa_to_discharge =   abs(li_ion.capa_to_discharge)
    
    # flow_peak =                  abs(flow.peak_discharge)
    # flow_capa_to_charge =        abs(flow.capa_to_charge)
    # flow_capa_to_discharge =     abs(flow.capa_to_discharge)
   
    
    # flywheel_peak =              abs(flywheel.peak_discharge)
    # flywheel_capa_to_charge =    abs(flywheel.capa_to_charge)
    # flywheel_capa_to_discharge = abs(flywheel.capa_to_discharge)
    
    soc_list = {'li_ion_soc': li_ion_soc, 'flow_soc': flow_soc, 'flywheel_soc': flywheel_soc}
    soc_list = sorted(soc_list, key=lambda x: (soc_list[x]))
    
    if action == 0:
        # NON-ISLANDED OVERGENERATION EXPORT
        control_dict = {    'pv_consummed': min(pv,load),
                            'li_charge': 0,
                            'li_discharge': 0,
                            'flow_charge': 0,
                            'flow_discharge': 0,
                            'flywheel_charge': 0,
                            'flywheel_discharge': 0,
                            'grid_import': 0,
                            'grid_export': overgen_net_load,
                            'genset': 0
                        }
                        
        return control_dict

########################################################################################################################
    if action == 1:
        # NON-ISLANDED OVERGENERATION EXCLUSIVE CHARGE AND EXPORT
        # print(soc_list[0])
        if soc_list[0] == 'li_ion_soc':
            # Charge li-ion exclusively
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': li_ion_charge_max,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': overgen_net_load-min(pv-load,li_ion_charge_max),
                                'genset': 0
                            }

        elif soc_list[0] == 'flow_soc':
            # Charge flow exclusively
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': flow_charge_max,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': overgen_net_load-min(pv-load,flow_charge_max),
                                'genset': 0
                            }
            

        elif soc_list[0] == 'flywheel_soc':
            # Charge flywheel exclusively
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': flywheel_charge_max,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': overgen_net_load-min(pv-load,flywheel_charge_max),
                                'genset': 0
                            }

                            
        return control_dict

########################################################################################################################
    if action == 2:
        # ISLANDED OVERGENERATION EXCLUSIVE CHARGE
        if soc_list[0] == 'li_ion_soc':
            # Charge li-ion exclusively
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': min(overgen_net_load,li_ion_charge_max),
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }

        elif soc_list[0] == 'flow_soc':
            # Charge flow exclusively
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': min(overgen_net_load,flow_charge_max),
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }

        elif soc_list[0] == 'flywheel_soc':
            # Charge flywheel exclusively
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': min(overgen_net_load,flywheel_charge_max),
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }
        return control_dict
                        
########################################################################################################################
    elif action == 3:
        # NON-ISLANDED OVERGENERATION COMBINED CHARGE
        control_dict = {    'pv_consummed': min(pv,load),
                            'li_charge': min((overgen_net_load)/3,li_ion_charge_max),
                            'li_discharge': 0,
                            'flow_charge': min((overgen_net_load)/3,flow_charge_max),
                            'flow_discharge': 0,
                            'flywheel_charge': min((overgen_net_load)/3,flywheel_charge_max),
                            'flywheel_discharge': 0,
                            'grid_import': 0,
                            'grid_export': max(0, (overgen_net_load - li_ion_charge_max - flow_charge_max - flywheel_charge_max)),
                            'genset': 0
                           }
        return control_dict

########################################################################################################################
    elif action == 4:
        # ISLANDED OVERGENERATION COMBINED CHARGE
        control_dict = {    'pv_consummed': min(pv,load),
                            'li_charge': min((overgen_net_load)/3,li_ion_charge_max),
                            'li_discharge': 0,
                            'flow_charge': min((overgen_net_load)/3,flow_charge_max),
                            'flow_discharge': 0,
                            'flywheel_charge': min((overgen_net_load)/3,flywheel_charge_max),
                            'flywheel_discharge': 0,
                            'grid_import': 0,
                            'grid_export':0,
                            'genset': 0
                           }
        return control_dict

########################################################################################################################
    elif action == 5:
        # NON-ISLANDED OVERGENERATION CONDITIONAL CHARGE
        control_dict = {        'pv_consummed': min(pv,load),
                                'li_charge': li_ion_charge_max,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': overgen_net_load-min(pv-load,li_ion_charge_max),
                                'genset': 0
                            }

        if li_ion_soc == 1:
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': flow_charge_max,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': overgen_net_load-min(pv-load,flow_charge_max),
                                'genset': 0
                            }
        elif (li_ion_soc, flow_soc) == (1,1):
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': flywheel_charge_max,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': overgen_net_load-min(pv-load,flywheel_charge_max),
                                'genset': 0
                            }
        return control_dict

########################################################################################################################
    elif action == 6:
        # ISLANDED OVERGENERATION CONDITIONAL CHARGE
        control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': li_ion_charge_max,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }

        if li_ion_soc == 1:
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': flow_charge_max,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }
        elif (li_ion_soc, flow_soc) == (1,1):
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': flywheel_charge_max,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }
        return control_dict

########################################################################################################################
    if action == 7:
        # NON-ISLANDED UNDERGENERATION EXCLUSIVE CHARGE
        
        # print(f'{(li_ion_soc, flow_soc, flywheel_soc)}')

                        
        if soc_list[0] == 'li_ion_soc':
            # Charge li-ion exclusively
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': li_ion_charge_max,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': undergen_net_load+li_ion_charge_max,
                                'grid_export': 0,
                                'genset': 0
                            }

        elif soc_list[0] == 'flow_soc':
            # Charge flow exclusively
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': flow_charge_max,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': undergen_net_load+flow_charge_max,
                                'grid_export': 0,
                                'genset': 0
                            }

        elif soc_list[0] == 'flywheel_soc':
            # print('charging flywheel')
            # Charge flywheel exclusively
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': flywheel_charge_max,
                                'flywheel_discharge': 0,
                                'grid_import': undergen_net_load+flywheel_charge_max,
                                'grid_export': 0,
                                'genset': 0
                            }
        return control_dict

########################################################################################################################
# Don't forget to add peaking power logic here
    if action == 8:
        # ISLANDED UNDERGENERATION EXCLUSIVE DISCHARGE
        if soc_list[2] == 'li_ion_soc': # Check if Li ion battery is the highest charged storage device
            # Discharge li-ion exclusively
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': min(undergen_net_load,li_ion_discharge_max),
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }
            if li_ion_discharge_max < undergen_net_load: # Use Genset
                control_dict = {'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': min(undergen_net_load,li_ion_discharge_max),
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 1
                                }

        elif soc_list[2] == 'flow_soc': # Check if flow battery is the highest charged storage device
            # Discharge flow exclusively
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': min(undergen_net_load,flow_discharge_max),
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }
            if flow_discharge_max < undergen_net_load: # Use Genset
                control_dict = {'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': min(undergen_net_load,flow_discharge_max),
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 1
                                }

        elif soc_list[2] == 'flywheel_soc': # Check if flywheel is the highest charged storage device
            # Discharge flywheel exclusively
            control_dict = {    'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': min(undergen_net_load,flywheel_charge_max),
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }
            if flow_discharge_max < undergen_net_load: # Use Genset
                control_dict = {'pv_consummed': min(pv,load),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': min(undergen_net_load, flywheel_discharge_max),
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 1
                                }
        return control_dict

# ########################################################################################################################
#     elif action == 9:
#         # NON-ISLANDED UNDERGENERATION COMBINED DISCHARGE
#         # Sell leftover power at night in non-disaster settings
#         control_dict = {        'pv_consummed': min(pv,load),
#                                 'li_charge': 0,
#                                 'li_discharge': min(undergen_net_load/3, li_ion_discharge_max),
#                                 'flow_charge': 0,
#                                 'flow_discharge': min(undergen_net_load/3, flow_discharge_max),
#                                 'flywheel_charge': 0,
#                                 'flywheel_discharge': min(undergen_net_load/3, flywheel_discharge_max),
#                                 'grid_import': 0,
#                                 'grid_export': max(0,(li_ion_discharge_max + flow_discharge_max + flywheel_discharge_max)-undergen_net_load),
#                                 'genset': 0
#                             }
#         if (li_ion_soc,flow_soc,flywheel_soc) == (0, 0, 0):
#             control_dict = {    'pv_consummed': min(pv,load),
#                                 'li_charge': 0,
#                                 'li_discharge': 0,
#                                 'flow_charge': 0,
#                                 'flow_discharge': 0,
#                                 'flywheel_charge': 0,
#                                 'flywheel_discharge': 0,
#                                 'grid_import': undergen_net_load,
#                                 'grid_export': 0,
#                                 'genset': 0
#                             }
#          return control_dict

# ########################################################################################################################
#     elif action == 9:
#         # ISLANDED UNDERGENERATION COMBINED DISCHARGE
#         control_dict = {        'pv_consummed': min(pv,load),
#                                 'li_charge': 0,
#                                 'li_discharge': min(undergen_net_load/3, li_ion_discharge_max),
#                                 'flow_charge': 0,
#                                 'flow_discharge': min(undergen_net_load/3, flow_discharge_max),
#                                 'flywheel_charge': 0,
#                                 'flywheel_discharge': min(undergen_net_load/3, flywheel_discharge_max),
#                                 'grid_import': 0,
#                                 'grid_export': 0,
#                                 'genset': 0
#                             }
#         if (li_ion_soc,flow_soc,flywheel_soc) == (0, 0, 0):
#             control_dict = {    'pv_consummed': min(pv,load),
#                                 'li_charge': 0,
#                                 'li_discharge': 0,
#                                 'flow_charge': 0,
#                                 'flow_discharge': 0,
#                                 'flywheel_charge': 0,
#                                 'flywheel_discharge': 0,
#                                 'grid_import': 0,
#                                 'grid_export': 0,
#                                 'genset': 1
#                             } 
#         return control_dict


   

In [11]:
def change_name_action(idx):
# Grid is up
    if idx == 0:
        action_name = "NON-ISLANDED OVERGENERATION EXPORT"
    elif idx == 1:
        action_name = "NON-ISLANDED OVERGENERATION EXCLUSIVE CHARGE AND EXPORT"
    elif idx == 2:
        action_name = "ISLANDED OVERGENERATION EXCLUSIVE CHARGE"
    elif idx == 3:
        action_name = "NON-ISLANDED OVERGENERATION COMBINED CHARGE"
    elif idx == 4:
        action_name = "ISLANDED OVERGENERATION COMBINED CHARGE"
    elif idx == 5:
        action_name = "NON-ISLANDED OVERGENERATION CONDITIONAL CHARGE"
    elif idx == 6 :
        action_name = "ISLANDED OVERGENERATION CONDITIONAL CHARGE"
    elif idx == 7 :
        action_name = "NON-ISLANDED UNDERGENERATION EXCLUSIVE CHARGE"
    elif idx == 8 :
        action_name = "ISLANDED UNDERGENERATION EXCLUSIVE DISCHARGE"
    # elif idx == 9 :
    #     action_name = "NON-ISLANDED UNDERGENERATION COMBINED DISCHARGE"
    # elif idx == 9 :
    #     action_name = "ISLANDED UNDERGENERATION COMBINED DISCHARGE"
    # elif idx == 11 :
    #     action_name = "grid down, discharge, battery < full, pv > load"
    # print(idx)
    return action_name

In [12]:
# def init_qtable(mg0, nb_action):
#     li_ion, flow, flywheel = mg0.ss.unpack()
#     cover_rate = (mg0._pv_ts*100)/mg0._load_ts
#     cover_rate_range = np.arange(int(cover_rate.min()),int(cover_rate.max()))
#     li_ion_soc_range = np.arange(round(li_ion.MIN_SOC,2),round(li_ion.MAX_SOC+0.05,2),0.05)
#     flow_soc_range = np.arange(round(flow.MIN_SOC,2),round(flow.MAX_SOC+0.05,2),0.05)
#     flywheel_soc_range = np.arange(round(flywheel.MIN_SOC,2),round(flywheel.MAX_SOC+0.05,2),0.05)
#     Q = {}
#     for idx1 in cover_rate_range:
#         Q[idx1] = {}
#         for idx2 in li_ion_soc_range:
#             Q[idx1][round(idx2,1)] = {}
#             for idx3 in flow_soc_range:
#                 Q[idx1][round(idx2,1)][round(idx3,1)] = {}
#                 for idx4 in flywheel_soc_range:
#                     Q[idx1][round(idx2,1)][round(idx3,1)][round(idx4,1)] = {}
#                     for action_idx in range(nb_action):
#                         Q[idx1][round(idx2,1)][round(idx3,1)][round(idx4,1)][action_idx] = 0 # Initialize all actions at 0
#     return Q
    

In [13]:
def init_qtable(mg0, nb_action):
    state = []
    Q = {}
    li_ion, flow, flywheel = mg0.ss.unpack()
    cover_rate = (mg0._pv_ts*100)/mg0._load_ts
    cover_rate_range = np.arange(int(cover_rate.min()),int(cover_rate.max()))
    li_ion_soc_range = np.arange(round(li_ion.MIN_SOC,1),round(li_ion.MAX_SOC+0.1,1),0.1)
    flow_soc_range = np.arange(round(flow.MIN_SOC,1),round(flow.MAX_SOC+0.1,1),0.1)
    flywheel_soc_range = np.arange(round(flywheel.MIN_SOC,1),round(flywheel.MAX_SOC+0.1,1),0.1)
    
    for idx1 in cover_rate_range:
        for idx2 in li_ion_soc_range:
            for idx3 in flow_soc_range:
                for idx4 in flywheel_soc_range:
                    idx4 = round(idx4,2)
                    idx2 = round(idx2,2)
                    idx3 = round(idx3,2)
                    state.append((idx1,idx2,idx3,idx4))

    for s in state:
        Q[s] = {}
        for a in range(nb_action):
            Q[s][a] = 0

    return Q
    

In [11]:
control_dict = {                'pv_consummed': 5.5,
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 3.3,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }
display(control_dict.values())
new = list(control_dict.values())
print(new[0])
print(new)

dict_values([5.5, 0, 0, 0, 0, 3.3, 0, 0, 0, 0])

5.5
[5.5, 0, 0, 0, 0, 3.3, 0, 0, 0, 0]


In [14]:
Q1 = init_qtable(mg0,11)
print(len(Q1))


120000


In [15]:
# print(Q1[(0,0.65,0.62,0.0)])

I define a function for my exploration strategy. Here I've selected the epsilon greedy decreasing strategy

In [16]:
def espilon_decreasing_greedy(action, epsilon, nb_action):
    p = np.random.random()
    if p < (1 - epsilon):
        randomm=0
        return action, randomm
    else: 
        randomm=1
        return np.random.choice(nb_action), randomm

In [17]:
def max_dict(d):

    max_key = None
    max_val = float('-inf')


    for k,v in d.items():

        if v > max_val:

            max_val = v
            max_key = k

    return max_key, max_val

I also need to define a function to decrease and update my espilon 

In [18]:
def update_epsilon(epsilon):
    
    epsilon = epsilon - epsilon *0.02
    
    if epsilon < 0.1:
        
        epsilon = 0.1
    
    return epsilon

In [19]:
def print_welcome(idx):
    
    if idx == 0:
        print("------------------------------------")
        print("|        WELCOME TO PYMGRID        |")
        print("------------------------------------")
    elif idx == 1:
        
        print("t -     STATE  -  ACTION - COST")
        print("================================")
    

Now I define my agent function

In [28]:
def training_Q_Learning(mg,horizon):
    li_ion,flow,flywheel = mg.ss.unpack()
    nb_action = 9
    Q = init_qtable(mg,nb_action)
    nb_state = len(Q)
    nb_episode = int(50000)
    alpha = 0.99
    epsilon = 0.99
    gamma = 0.99
    li_ion.soc = 1
    flow.soc = 1
    flywheel.soc = 1
    record_cost = []
    t0 = time.time()
    t = t0
    print_training = "Training Progressing .   "
    print_welcome(0)
    print("\n")
    for e in range(nb_episode+1):
        
        if e == 0:
            value_print="\rEpisode " + str(e) +"/" + str(nb_episode)
            sys.stdout.write(value_print)
            sys.stdout.flush()
        
        else:
            now = time.time()
            if e == 1 or e == 50 or e == 100 or e == 200 or e == 300 or e == 400  or e == 450  or e == 485 or e == nb_episode-5:
                diff_time_t = now - t 
                total_time = (nb_episode+1) * diff_time_t
            
            now = time.time()
            
            diff_time = now - t0
            time_remaining = total_time - diff_time
            
            t = time.time()
        
        if e % 10 == 0:
        
            if print_training == "Training Progressing .   ":
                
                print_training = "Training Progressing ..  "
                
            elif print_training == "Training Progressing ..  ":
            
                print_training = "Training Progressing ... "
            
            else:
                
                print_training = "Training Progressing .   "

        value_print="\r"+ print_training +"Episode " + str(e) +"/" + str(nb_episode) 
        sys.stdout.write(value_print)
        sys.stdout.flush()
            
        episode_cost = 0
        mg.reset()
        
        cover_rate = int((mg.pv*100)/mg.load)
        li_ion_soc = round(li_ion.soc,1)
        flow_soc = round(flow.soc,1)
        flywheel_soc = round(flywheel.soc,1)
        # if flow_soc > 1.0 :
        #     flow_soc = 1
        # if flywheel_soc > 1.0:
        #     flywheel_soc = 1
        # if li_ion_soc > 1.0:    
        #     li_ion_soc = 1
        s = (cover_rate, li_ion_soc, flow_soc, flywheel_soc)
        a = max_dict(Q[s])[0]
        a,randomm = espilon_decreasing_greedy(a, epsilon, nb_action)
        for i in range (horizon):
            action = actions_agent(mg,a)
            action_name = change_name_action(a)
            status = mg.run(action)
            r = -mg.get_cost() 
            episode_cost += mg.get_cost()/3600
            cover_rate = int((mg.pv*100)/mg.load)
            li_ion_soc = round(li_ion.soc,1)
            flow_soc = round(flow.soc,1)
            flywheel_soc = round(flywheel.soc,1)
            if flow_soc < 0 or flywheel_soc < 0 or li_ion_soc < 0:
                raise ValueError(f'li: {li_ion_soc}, flow: {flow_soc}, flywheel {flywheel_soc}, action: '+action_name+f'{action}')
            s_ = (cover_rate, li_ion_soc, flow_soc, flywheel_soc)
            a_ = max_dict(Q[s_])[0]
            if i == horizon-1:
                Q[s][a] += alpha*(r - Q[s][a])
            else:
                old_Q = Q[s][a]
                target = r + gamma*Q[s_][a_]
                td_error = target - Q[s][a]
                Q[s][a] = (1-alpha) * Q[s][a] + alpha * td_error
            s, a = s_, a_
       
        epsilon = update_epsilon(epsilon)
        
    return Q

In [21]:
def testing_Q_Learning(mg, Q,horizon):
    li_ion,flow,flywheel = mg.ss.unpack()
    cost_record = np.array((1,))
    mg.reset()
    cover_rate = int((mg.pv*100)/mg.load)
    li_ion.soc = 1
    flow.soc = 1
    flywheel.soc = 1
    s = (cover_rate, li_ion.soc, flow.soc, flywheel.soc)
    a = max_dict(Q[s])[0]
    total_cost = 0
    print_welcome(1)
    export_counter = 0
    for i in range (1,horizon):
        li_ion_soc = round(li_ion.soc,1)
        flow_soc = round(flow.soc,1)
        flywheel_soc = round(flywheel.soc,1)
        # if flow_soc > 1.0 :
        #     flow_soc = 1
        # if flywheel_soc > 1.0:
        #     flywheel_soc = 1
        # if li_ion_soc > 1.0:    
        #     li_ion_soc = 1
        action_name = change_name_action(a)
        action = actions_agent(mg, a)
        status = mg.run(action)
        # display(status) 
        cost = mg.get_cost()/3600
        cost_record = np.append(cost_record, cost+cost_record[i-1])
        total_cost += cost
        # if a == 0 or 2:
        #     export_counter += 1
        #     print(mg.get_cost()/3600)
        if i < 10:
            print(i," -",f'{a}')
        else:
            print(i,"-",f'{a}')

        # if i < 10:
        #     print(i," -",f'cr: {int(mg.pv)},li: {li_ion_soc}, flow: {flow_soc}, flywheel: {flywheel_soc}',action_name, round(total_cost,2), "$")
        # else:
        #     print(i,"-",f'cr: {int(mg.pv)},li: {li_ion_soc}, flow: {flow_soc}, flywheel: {flywheel_soc}',action_name, round(total_cost,2), "$")
        cover_rate = int((mg.pv*100)/mg.load)
        
        s_ = (cover_rate, li_ion_soc, flow_soc, flywheel_soc)
        a_ = max_dict(Q[s_])[0]

        s, a = s_, a_
    return cost_record,export_counter

In [29]:
trained_Q1 = training_Q_Learning(mg0,100)
with open("myfile.txt", 'w') as f: 
    for key, value in trained_Q1.items():
        f.write('%s:%s\n' % (key, value))

------------------------------------
|        WELCOME TO PYMGRID        |
------------------------------------


Training Progressing .   Episode 50000/50000

In [23]:
with open("myfile.txt", 'w') as f: 
    for key, value in trained_Q1.items():
        f.write('%s:%s\n' % (key, value))

In [30]:
cost_record,export_counter = testing_Q_Learning(mg1,trained_Q1, DAY)

t -     STATE  -  ACTION - COST
1  - 7
2  - 7
3  - 7
4  - 7
5  - 7
6  - 7
7  - 7
8  - 7
9  - 7
10 - 7
11 - 7
12 - 7
13 - 7
14 - 7
15 - 7
16 - 7
17 - 7
18 - 7
19 - 7
20 - 7
21 - 7
22 - 7
23 - 7
24 - 7
25 - 7
26 - 7
27 - 7
28 - 7
29 - 7
30 - 7
31 - 7
32 - 7
33 - 7
34 - 7
35 - 7
36 - 7
37 - 7
38 - 7
39 - 7
40 - 7
41 - 7
42 - 7
43 - 7
44 - 7
45 - 7
46 - 7
47 - 7
48 - 7
49 - 7
50 - 7
51 - 7
52 - 7
53 - 7
54 - 7
55 - 7
56 - 7
57 - 7
58 - 7
59 - 7
60 - 7
61 - 7
62 - 7
63 - 7
64 - 7
65 - 7
66 - 7
67 - 7
68 - 7
69 - 7
70 - 7
71 - 7
72 - 7
73 - 7
74 - 7
75 - 7
76 - 7
77 - 7
78 - 7
79 - 7
80 - 7
81 - 7
82 - 7
83 - 7
84 - 7
85 - 7
86 - 7
87 - 7
88 - 7
89 - 7
90 - 7
91 - 7
92 - 7
93 - 7
94 - 7
95 - 7
96 - 7
97 - 7
98 - 7
99 - 7
100 - 7
101 - 7
102 - 7
103 - 7
104 - 7
105 - 7
106 - 7
107 - 7
108 - 7
109 - 7
110 - 7
111 - 7
112 - 7
113 - 7
114 - 7
115 - 7
116 - 7
117 - 7
118 - 7
119 - 7
120 - 7
121 - 7
122 - 7
123 - 7
124 - 7
125 - 7
126 - 7
127 - 7
128 - 7
129 - 7
130 - 7
131 - 7
132 - 7
133 - 7
134

In [25]:
with open("myfile.txt", 'w') as f: 
    for key, value in Q1.items(): 
        f.write('%s:%s\n' % (key, value))

In [26]:
print(mg0.grid.power_import)

698684


In [27]:
print(sum(mg0._load_ts[0:DAY]))
arr = np.array(mg0._load_ts[0:DAY])
print(sum(arr)/3600)


0
[4047852.2373469]
